In [1]:
import json, pprint, requests, textwrap, time
host = 'http://150.140.142.92:8998'

In [2]:
#get number of sessions
headers = {'Content-Type': 'application/json'}
r = requests.get(host+'/sessions', headers=headers)
con_var=r.json()['total']
print ("Waiting... active sesssions: ", con_var)
while con_var!=0:
    r = requests.get(host+'/sessions', headers=headers)
    con_var=r.json()['total']
    print ("Waiting... active sesssions: ", con_var)
    time.sleep(60)
print ("Good to go!")

Waiting... active sesssions:  0
Good to go!


In [3]:
#create new session - DON'T DO THIS IF THERE ARE OTHER ACTIVE SESSIONS!
data = {'kind': 'pyspark',\
        'name':'Melina' ,
        'conf':{'master':'spark://localhost:7077'}\
       }#, 'heartbeatTimeoutInSecond':240}

r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{'id': 172,
 'name': 'Melina',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['stdout: ', '\nstderr: ']}

In [4]:
#get session information
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{'id': 172,
 'name': 'Melina',
 'appId': None,
 'owner': None,
 'proxyUser': None,
 'state': 'starting',
 'kind': 'pyspark',
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'log': ['21/06/04 23:52:25 INFO BlockManager: Using org.apache.spark.storage.RandomBlockReplicationPolicy for block replication policy',
  '21/06/04 23:52:25 INFO StandaloneAppClient$ClientEndpoint: Executor updated: app-20210604235225-0156/0 is now RUNNING',
  '21/06/04 23:52:25 INFO StandaloneAppClient$ClientEndpoint: Executor updated: app-20210604235225-0156/1 is now RUNNING',
  '21/06/04 23:52:25 INFO BlockManagerMaster: Registering BlockManager BlockManagerId(driver, 150.140.142.92, 33455, None)',
  '21/06/04 23:52:25 INFO BlockManagerMasterEndpoint: Registering block manager 150.140.142.92:33455 with 366.3 MB RAM, BlockManagerId(driver, 150.140.142.92, 33455, None)',
  '21/06/04 23:52:25 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, 150.140.142.92, 33455, None)',
  '21/06/04 

In [5]:
#show session state, repeat until idle to proceed with code submission
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is idle
Good to go!


In [9]:
#send some python statements for execution
codefile = open('LivyTestQuery3.py', 'r')
code = codefile.read()
codefile.close()
data = {
    'kind': 'pyspark',
    'code': textwrap.dedent(code)
}

statement_url = session_url+'/statements'
r = requests.post(statement_url, data=json.dumps(data), headers=headers)

In [10]:
#show session state, repeat until idle to proceed with result extraction
state_url = session_url + '/state'
con_var=".."
while con_var!="idle":
    r = requests.get(state_url, headers=headers)
    con_var = r.json()['state']
    print("Job is %s" % r.json()['state'])
    time.sleep(10)
print ("Good to go!")

Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is busy
Job is idle
Good to go!


In [11]:
#get stats from the latest statement in the session
statements_url = session_url + '/statements'
print(statements_url)
r = requests.get(statements_url, headers=headers)
slist = r.json()
last_statement = slist['statements'][slist['total_statements']-1]
if last_statement['progress']==1:
    print ("Job completed")

print("Total Time %s ms" % (last_statement['completed'] - last_statement['started']))

http://150.140.142.92:8998/sessions/172/statements
Job completed
Total Time 122086 ms


In [38]:
#get last statement output
output = last_statement['output']
if output['status']=="error":
    pprint.pprint(output['traceback'])
else:
    fres = output['data']['text/plain'].replace("'","")
    #pprint.pprint(output['data']['text/plain'])
    fres = json.loads(fres)
print(fres[0]['author'])

TypeError: 'int' object is not subscriptable

In [12]:
#close the session
r = requests.delete(session_url, headers=headers)
pprint.pprint(r.json())


{'msg': 'deleted'}


In [72]:
#close an arbitrary session
sessionid = 109
r = requests.delete("http://150.140.142.92:8998/sessions/"+str(sessionid), headers={'Content-Type': 'application/json'})
pprint.pprint(r.json())

{'msg': "Session '109' not found."}
